### 문제 7-1 : LangGraph ReAct Agent 실습 연습문제 (Vector DB + Tool 연동)

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import uuid

import warnings
warnings.filterwarnings("ignore")

In [3]:
from langgraph.graph import MessagesState

class AgentState(MessagesState):
    pass  # MessagesState는 대화 내역 저장 클래스, 그대로 사용

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_ollama  import OllamaEmbeddings
from langchain_core.tools import tool
from typing import List

embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 

# menu db 벡터 저장소 로드
cafe_db = FAISS.load_local(
    "../db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

# Tool 정의 
@tool
def search_cafe_menu(query: str) -> List[str]:
    """카페 메뉴에서 정보를 검색합니다."""
    docs = cafe_db.similarity_search(query, k=6)

    formatted_docs = "\n\n---\n\n".join(
        [
            f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )

    if len(docs) > 0:
        return formatted_docs
    
    return "관련 메뉴 정보를 찾을 수 없습니다."

In [7]:
from langchain_core.messages import SystemMessage
from langgraph.graph import StateGraph

def agent_node(state: AgentState):
    system_message = SystemMessage(content="당신은 카페 메뉴를 안내하는 AI입니다.")
    messages = [system_message] + state['messages']
    response = llm_with_tools.invoke(messages)  # LLM에 메시지 보내기
    return {"messages": [response]}


In [6]:
from langchain_openai import ChatOpenAI

# LLM 모델 
llm = ChatOpenAI(model="gpt-4o-mini", streaming=True)
print(llm.model_name)

# 도구 목록
tools = [search_cafe_menu]

# 모델에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=tools)
print(type(llm_with_tools))

gpt-4o-mini
<class 'langchain_core.runnables.base.RunnableBinding'>


In [8]:
from langgraph.prebuilt import ToolNode

tools = [search_cafe_menu]  # 툴 목록
tool_node = ToolNode(tools)


In [9]:
from langgraph.prebuilt import tools_condition

builder = StateGraph(AgentState)

# 노드 등록
builder.add_node("agent", agent_node)
builder.add_node("tools", tool_node)

# 조건부 분기: 도구 호출 필요 여부 자동 판단
builder.add_conditional_edges("agent", tools_condition)

# tools 실행 후 다시 agent로
builder.add_edge("tools", "agent")

# 시작, 끝 지정
builder.set_entry_point("agent")
builder.set_finish_point("agent")

# 그래프 만들기
graph = builder.compile()


In [13]:
from langchain_core.messages import HumanMessage
inputs = {"messages": [HumanMessage(content="아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.")]}
result = graph.invoke(inputs)

for msg in result["messages"]:
    print(msg.content)



아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.

<Document source="../data/cafe_menu_data.txt"/>
1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.
</Document>

---

<Document source="../data/cafe_menu_data.txt"/>
9. 아이스 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 차가운 물, 얼음
   • 설명: 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가 높습니다.
</Document>

---

<Document source="../data/cafe_menu_data.txt"/>
3. 카푸치노
   • 가격: ₩5,000
   • 주요 원료: 에스프레소, 스팀 밀크, 우유 거품
   • 설명: 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 일품이며, 계피 파우더를 뿌려 제공합니다.
</Document>

---

<Document source="../data/cafe_menu_data.txt"/>
7. 프라푸치노
   • 가격: ₩7,000
   • 주요 원료: 에스프레소, 우유, 얼음, 휘핑크림
   • 설명: 에스프레소와 우유, 얼음을 블렌더에 갈아 만든 시원한 음료입니다. 부드럽고 크리미한 질감이 특징이며, 휘핑크림을 올려 달콤함을 더했습니다. 여름철 인기 메뉴입니다.
</Document>

---

<Document source="../data/cafe_menu_data.txt"/>
10. 티라

In [14]:
inputs = {"messages": [HumanMessage(content="라떼 종류에는 어떤 메뉴들이 있고 각각의 특징은 무엇인가요?")]}
result = graph.invoke(inputs)

for msg in result["messages"]:
    print(msg.content)

라떼 종류에는 어떤 메뉴들이 있고 각각의 특징은 무엇인가요?

<Document source="../data/cafe_menu_data.txt"/>
4. 바닐라 라떼
   • 가격: ₩6,000
   • 주요 원료: 에스프레소, 스팀 밀크, 바닐라 시럽
   • 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.
</Document>

---

<Document source="../data/cafe_menu_data.txt"/>
8. 녹차 라떼
   • 가격: ₩5,800
   • 주요 원료: 말차 파우더, 스팀 밀크, 설탕
   • 설명: 고급 말차 파우더와 부드러운 스팀 밀크로 만든 건강한 음료입니다. 녹차의 은은한 쓴맛과 우유의 부드러움이 조화를 이루며, 항산화 성분이 풍부합니다. 달콤함 조절이 가능합니다.
</Document>

---

<Document source="../data/cafe_menu_data.txt"/>
2. 카페라떼
   • 가격: ₩5,500
   • 주요 원료: 에스프레소, 스팀 밀크
   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 특징이며, 다양한 시럽과 토핑 추가가 가능합니다. 라떼 아트로 시각적 즐거움도 제공합니다.
</Document>

---

<Document source="../data/cafe_menu_data.txt"/>
5. 카라멜 마키아토
   • 가격: ₩6,500
   • 주요 원료: 에스프레소, 스팀 밀크, 카라멜 시럽, 휘핑크림
   • 설명: 스팀 밀크 위에 에스프레소를 부어 만든 후 카라멜 시럽과 휘핑크림으로 마무리한 달콤한 커피입니다. 카라멜의 진한 단맛과 커피의 깊은 맛이 조화를 이루며, 시각적으로도 아름다운 층을 형성합니다.
</Document>

---

<Document source="../da

In [15]:
inputs = {"messages": [HumanMessage(content="디저트 메뉴 중에서 티라미수에 대해 자세히 설명해주세요.")]}
result = graph.invoke(inputs)

for msg in result["messages"]:
    print(msg.content)

디저트 메뉴 중에서 티라미수에 대해 자세히 설명해주세요.

<Document source="../data/cafe_menu_data.txt"/>
10. 티라미수
    • 가격: ₩7,500
    • 주요 원료: 마스카포네 치즈, 에스프레소, 레이디핑거, 코코아 파우더
    • 설명: 이탈리아 전통 디저트로 마스카포네 치즈와 에스프레소에 적신 레이디핑거를 층층이 쌓아 만들었습니다. 부드럽고 달콤한 맛이 특징이며, 코코아 파우더로 마무리하여 깊은 풍미를 자랑합니다.
</Document>

---

<Document source="../data/cafe_menu_data.txt"/>
5. 카라멜 마키아토
   • 가격: ₩6,500
   • 주요 원료: 에스프레소, 스팀 밀크, 카라멜 시럽, 휘핑크림
   • 설명: 스팀 밀크 위에 에스프레소를 부어 만든 후 카라멜 시럽과 휘핑크림으로 마무리한 달콤한 커피입니다. 카라멜의 진한 단맛과 커피의 깊은 맛이 조화를 이루며, 시각적으로도 아름다운 층을 형성합니다.
</Document>

---

<Document source="../data/cafe_menu_data.txt"/>
9. 아이스 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 차가운 물, 얼음
   • 설명: 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가 높습니다.
</Document>

---

<Document source="../data/cafe_menu_data.txt"/>
2. 카페라떼
   • 가격: ₩5,500
   • 주요 원료: 에스프레소, 스팀 밀크
   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 특징이며, 다양한 시럽과 토핑 추가가 가능합니다. 라떼 아트로 시각적 즐거움도 제공합니다.
</Document>

---

